# IMPORT

In [1]:
import torch
import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/") # PATH TO PYSEQM
import seqm
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz
seqm.seqm_functions.scf_loop.debug=True

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Ti-Fe

In [22]:
seqm.seqm_functions.scf_loop.MAX_ITER=1000


species, coordinates = read_xyz(['dimers_.xyz', 'dimers_stretched.xyz', 
                                ])
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]



const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [1, 0.9, 0.9, 100], # converger used for scf loop
                                           # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                           # [1], adaptive mixing
                                           # [1, K, L, M] # advanced adaptive mixing.
                                           # First, it does linear mixing for M steps. Mixing coeff is K the first 5 SCF steps.
                                           # Then it incrementally goes to L and becomes L at M-5 SCF step. From M-5 to M it's equal to L. After M'th SCF step, adaptive mixing begins.
                                           # 
                                           # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                    "Hf_flag": False,
                    'scf_backward' : 0,
                    'UHF' : False,
                   }

#charges = torch.tensor([0],dtype=torch.int64, device=device)
molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)
esdriver = Electronic_Structure(seqm_parameters).to(device)
esdriver(molecules, 
         #P0=dm
        )

scf direct step  :   0 | MAX ΔE[   1]:   42.3394197 | MAX ΔDM[   0]:    0.0235755 | MAX ΔDM_ij[   0]:  0.2000000  | N not converged: 2
scf direct step  :   1 | MAX ΔE[   0]:   31.4860713 | MAX ΔDM[   0]:    0.0212200 | MAX ΔDM_ij[   1]:  0.1799995  | N not converged: 2
scf direct step  :   2 | MAX ΔE[   0]:   23.0217737 | MAX ΔDM[   0]:    0.0191023 | MAX ΔDM_ij[   1]:  0.1619971  | N not converged: 2
scf direct step  :   3 | MAX ΔE[   0]:   16.4668576 | MAX ΔDM[   0]:    0.0172053 | MAX ΔDM_ij[   1]:  0.1457925  | N not converged: 2
scf direct step  :   4 | MAX ΔE[   0]:   11.5119164 | MAX ΔDM[   1]:    0.0155993 | MAX ΔDM_ij[   1]:  0.1312049  | N not converged: 2
scf direct step  :   5 | MAX ΔE[   1]:    9.5740820 | MAX ΔDM[   1]:    0.0148932 | MAX ΔDM_ij[   0]:  0.1399812  | N not converged: 2
scf direct step  :   6 | MAX ΔE[   0]:    7.9653871 | MAX ΔDM[   1]:    0.0157232 | MAX ΔDM_ij[   1]:  0.1329774  | N not converged: 2
scf direct step  :   7 | MAX ΔE[   0]:    8.0522933 | M

In [10]:
dm = molecules.dm.clone()

In [35]:
# Absolute energies
print('Eelec:',molecules.Eelec[0].item(),molecules.Eelec[1].item())
# MOPAC: -1563.9357  -1598.8704
# NEXMD: -1569.54353790   -1604.52140895

Eelec: -1563.9238871020257 -1598.9031236466196


In [36]:
# Absolute energies
print('Eelec:',molecules.Eelec[0].item(),molecules.Eelec[1].item())
# MOPAC: -1563.9357  -1598.8704
# NEXMD: -1569.54353790   -1604.52140895

Eelec: -1563.9238871020257 -1598.9031236466196


In [37]:
# Energy differences
print('Eelec:', 'PYSEQM', molecules.Eelec[0].item()-molecules.Eelec[1].item(),  'MOPAC', -1563.9357  +1598.8704, 'NEXMD', -1569.54353790   +1604.52140895)
# MOPAC: -1563.9357  -1598.8704
# NEXMD: -1569.54353790 

Eelec: PYSEQM 34.97923654459396 MOPAC 34.93470000000002 NEXMD 34.977871049999976
